In [1]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

def inite(elec,orb):
    config=[]
    list1=[]
    list1.append([e for e in range(elec)])
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)
            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:    
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)
                        list1.append(config)
                        config=[]
    return list1

def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=charge)
    hf_state = qml.qchem.hf_state(electrons, qubits)
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

    wires=range(qubits)
    params = np.zeros(len(singles) + len(doubles))
    # Define the device
    dev = qml.device("default.qubit", wires=qubits, shots=shots)
    
    # Define the qnode
    if shots==None:
        @qml.qnode(dev, interface="autograd", diff_method="adjoint")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
    else:
        @qml.qnode(dev, interface="autograd")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)

    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(circuit, params, wires=range(qubits),
                                                 s_wires=s_wires, d_wires=d_wires,
                                                 hf_state=hf_state)
    
    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params


# Define the molecule
r = 1.88973
symbols  = [ 'H', 'H']
geometry = np.array([[0.0, 0.0,  0.0],
                     [0.0, 0.0,  0.735*r]
                    
                     ], requires_grad = False)
electrons = 2
charge = 0

params = gs_exact(symbols,geometry,electrons,charge,max_iter=50)

/Users/prince.kwao/miniconda3/envs/cwp/lib/python3.13/site-packages/pennylane/ops/op_math/composite.py:211: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)


Ground state energy: -1.13730604 Ha


In [2]:
def qsceom(symbols, geometry, electrons, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)

    null_state = np.zeros(qubits,int)
    list1 = inite(electrons,qubits)
    values =[]

    if shots==0:
        #dev = qml.device("default.qubit", wires=qubits)
        dev = qml.device("default.qubit", wires=qubits)
    else:
        #dev = qml.device("default.qubit", wires=qubits,shots=shots)
        dev = qml.device("default.qubit", wires=qubits,shots=shots)
    #circuit for diagonal part
    @qml.qnode(dev)
    def circuit_d(params, occ,wires, s_wires, d_wires, hf_state):
        for w in occ:
            qml.X(wires=w)
        qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
        return qml.expval(H)
    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2,wires, s_wires, d_wires, hf_state):
        for w in occ1:
            qml.X(wires=w)
        first=-1
        for v in occ2:
            if v not in occ1:
                if first==-1:
                    first=v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first,v])
        for v in occ1:
            if v not in occ2:
                if first==-1:
                    first=v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first,v])
        qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    #print("diagonal parts done")
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    #print("off diagonal terms done")
    #ERROR:not subtracting the gs energy
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values
eig = qsceom(symbols, geometry, electrons, charge, params,shots=0)
print('excited state energies:\n', eig)

excited state energies:
 [tensor([-1.13730604, -0.52461743, -0.16275486,  0.49505412], requires_grad=True)]


In [8]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


def inite(elec, orb):
    config = []
    list1 = []
    list1.append([e for e in range(elec)])
    for x in range(elec):
        count = orb - elec
        while count < orb:
            for e in range(elec):
                if x == e:
                    if x % 2 == 0:
                        config.append(count)
                        count = count + 2
                    else:
                        config.append(count + 1)
                        count = count + 2
                else:
                    config.append(e)
            list1.append(config)
            config = []
    # doubles
    for x in range(elec):
        for y in range(x + 1, elec):
            count1 = orb - elec
            count2 = orb - elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont = 0
                    if count1 == count2:
                        if (x % 2) != (y % 2):
                            cont = 1
                    else:
                        cont = 1
                    if (x % 2) == (y % 2) and count2 < count1:
                        cont = 0
                    if cont == 1:
                        for e in range(elec):
                            if x == e:
                                if x % 2 == 0:
                                    config.append(count1)
                                else:
                                    config.append(count1 + 1)
                            elif y == e:
                                if y % 2 == 0:
                                    config.append(count2)
                                else:
                                    config.append(count2 + 1)
                            else:
                                config.append(e)
                        list1.append(config)
                        config = []
    return list1


def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=charge)
    hf_state = qml.qchem.hf_state(electrons, qubits)
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

    wires = range(qubits)
    params = np.zeros(len(singles) + len(doubles))
    dev = qml.device("default.qubit", wires=qubits, shots=shots)

    if shots is None:
        @qml.qnode(dev, interface="autograd", diff_method="adjoint")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
    else:
        @qml.qnode(dev, interface="autograd")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)

    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params


r = 1.88973
symbols = ['H', 'H']
geometry = np.array([[0.0, 0.0, 0.0],
                     [0.0, 0.0, 0.735 * r]], requires_grad=False)
electrons = 2
charge = 0

params = gs_exact(symbols, geometry, electrons, charge, max_iter=50)




/Users/prince.kwao/miniconda3/envs/cwp/lib/python3.13/site-packages/pennylane/ops/op_math/composite.py:211: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)


Ground state energy: -1.13730604 Ha


In [9]:
def bit_flip_channel(p, wires):
    K0 = np.sqrt(1 - p) * np.eye(2)
    K1 = np.sqrt(p) * np.array([[0, 1], [1, 0]])
    qml.QubitChannel([K0, K1], wires=wires)

def qsceom(symbols, geometry, electrons, charge, params, shots=0, bitflip_prob=0.1):
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires = range(qubits)

    null_state = np.zeros(qubits, int)
    list1 = inite(electrons, qubits)
    values = []

    if shots == 0:
        dev = qml.device("default.qubit", wires=qubits)
    else:
        dev = qml.device("default.qubit", wires=qubits, shots=shots)

    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        for w in occ:
            qml.X(wires=w)
        for w in wires:
            bit_flip_channel(bitflip_prob, wires=w)
        qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
        return qml.expval(H)

    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        for w in occ1:
            qml.X(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for w in wires:
            bit_flip_channel(bitflip_prob, wires=w)
        qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
        return qml.expval(H)

    M = np.zeros((len(list1), len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i, i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)

    for i in range(len(list1)):
        for j in range(len(list1)):
            if i != j:
                Mtmp = circuit_od(params, list1[i], list1[j], wires, s_wires, d_wires, null_state)
                M[i, j] = Mtmp - M[i, i] / 2.0 - M[j, j] / 2.0

    eig, evec = np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

eig = qsceom(symbols, geometry, electrons, charge, params, shots=0, bitflip_prob=0.1)
print('excited state energies:\n', eig)

DeviceError: Operator QubitChannel(tensor([[0.9486833, 0.       ],
        [0.       , 0.9486833]], requires_grad=True), tensor([[0.        , 0.31622777],
        [0.31622777, 0.        ]], requires_grad=True), wires=[0]) not supported with default.qubit and does not provide a decomposition.